In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("df_ex1_clean.csv")
df = df.drop(columns="Unnamed: 0")
display(df)

,Label,Message,Word_count,Message_clean,Word_count_clean
0,ham,"Go until jurong point, crazy.. Available only ...",20,crazy available bugis great world buffet cine ...,10
1,ham,Ok lar... Joking wif u oni...,6,lar joke wif oni,4
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,free entry wkly comp win cup final tkts text r...,15
3,ham,U dun say so early hor... U c already then say...,11,early hor,2
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,nah think go usf life,5
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,30,time try contact pound prize claim easy minute...,9
5568,ham,Will ü b going to esplanade fr home?,8,go esplanade home,3
5569,ham,"Pity, * was in mood for that. So...any other s...",10,pity mood soany suggestion,4
5570,ham,The guy did some bitching but I acted like i'd...,26,guy bitching act like interested buy week give...,9


In [ ]:
from gensim.models import Word2Vec, FastText

# Remove qualquer linha onde 'Message_clean' seja NaN
df.dropna(subset=['Message_clean'], inplace=True)
sentences = [msg.split() for msg in df["Message_clean"]]

print("Formato de dados para os modelos:")
print(sentences)

model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=-1)
model_w2v.init_sims(replace=True)
print("Modelo Word2Vec treinado!")

model_ft = FastText(sentences, vector_size=100, window=5, min_count=1, workers=-1)
model_ft.init_sims(replace=True)
print("Modelo FastText treinado!")

similar_words_ft = model_ft.wv.most_similar('say', topn=3)
print(f"\nPalavras mais similares a 'say' (FastText): {similar_words_ft}")

print("\n--- Testando palavra fora do vocabulário: 'joker' ---")
try:
    vector_oov_w2v = model_w2v.wv['joker']
except KeyError:
    print("Word2Vec: ERRO! A palavra 'joker' não está no vocabulário.")

vector_oov_ft = model_ft.wv['joker']
print("FastText: conseguiu criar um vetor para 'joker'.")
print(f"Palavras mais similares a 'joker' (FastText): {model_ft.wv.most_similar('joker')}")


Formato de dados para os modelos:
[['crazy', 'available', 'bugis', 'great', 'world', 'buffet', 'cine', 'get', 'amore', 'wat'], ['lar', 'joke', 'wif', 'oni'], ['free', 'entry', 'wkly', 'comp', 'win', 'cup', 'final', 'tkts', 'text', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply'], ['early', 'hor'], ['nah', 'think', 'go', 'usf', 'life'], ['freemsg', 'hey', 'darling', 'week', 'word', 'like', 'fun', 'send'], ['brother', 'like', 'speak', 'treat', 'like', 'aids', 'patent'], ['request', 'oru', 'minnaminunginte', 'set', 'callertune', 'caller', 'press', 'copy', 'friend', 'callertune'], ['winner', 'value', 'network', 'customer', 'select', 'receivea', 'prize', 'reward', 'claim', 'claim', 'code', 'valid', 'hour'], ['mobile', 'month', 'entitle', 'update', 'late', 'colour', 'mobile', 'camera', 'free', 'mobile', 'update', 'free'], ['home', 'soon', 'want', 'talk', 'stuff', 'anymore', 'tonight', 'cry', 'today'], ['chance', 'win', 'cash', 'pound', 'txt', 'send', 'cost', 'tsandcs', 'apply', 

/tmp/ipykernel_8128/6178843.py:11: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_w2v.init_sims(replace=True)


Modelo FastText treinado!

Palavras mais similares a 'say' (FastText): [('sayy', 0.5167182683944702), ('says', 0.4418196678161621), ('litre', 0.4054400622844696)]

--- Testando palavra fora do vocabulário: 'joker' ---
FastText: SUCESSO! Conseguiu criar um vetor para 'joker'.
Palavras mais similares a 'joker' (FastText): [('joke', 0.5302622318267822), ('poker', 0.390997976064682), ('sucker', 0.3750991225242615), ('rush', 0.3442986011505127), ('passable', 0.3323966860771179), ('lesser', 0.3262452185153961), ('unbelievable', 0.3217867910861969), ('push', 0.321247398853302), ('xin', 0.3210022449493408), ('walike', 0.31648480892181396)]


/tmp/ipykernel_8128/6178843.py:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_ft.init_sims(replace=True)
